In [1]:
import mechanicalsoup as ms
import pandas as pd
import numpy as np


In [2]:
def cleaningdata(data):
    #Dropping Credit scores above 850 and less than 301
    data=data.drop(data['credit_score'].loc[(data['credit_score'] < 301) | (data['credit_score'] > 850)].index)
    data=data.dropna(subset=['credit_score'])
    data=data.dropna(subset=['first_payment_date'])
    data['fthb_flag'] = data['fthb_flag'].fillna("NA")
    #Dropping not applicable MSA data
    data=data.dropna(subset=['msa'])
    data['mortage_insurance_pct'] = data['mortage_insurance_pct'].fillna(0)
    data['no_of_units'] = data['no_of_units'].fillna(0)
    data['cltv'] = data['cltv'].fillna(0)
    data['dti_ratio'] = data['dti_ratio'].fillna(0)
    data['original_ltv'] = data['original_ltv'].fillna(0)
    data['ppm_flag'] = data['ppm_flag'].fillna("U")
    data['prop_type']=data['prop_type'].fillna('NA')
    data['loan_purpose']=data['loan_purpose'].fillna('NA')
    data = data.dropna(subset=['zipcode'])
    data['number_of_borrowers'] = data['number_of_borrowers'].fillna(1)
    data['super_conforming_flag'] = data['super_conforming_flag'].fillna("N")
    
    return data

In [3]:
def convertnumbersdata(data):
    data['fthb_flag'] = data['fthb_flag'].replace(['Y','N','NA'],[1,2,3])
    data['occupancy_status'] = data['occupancy_status'].replace(['I','O','S','P'],[1,2,3,4])
    data['channel'] = data['channel'].replace(['B','C','R','T'],[1,2,3,4])
    data['ppm_flag'] = data['ppm_flag'].replace(['Y','N','U'],[1,2,3])
    data['prop_type'] = data['prop_type'].replace(['CO','LH','PU','MH','SF','CP','99'],[1,2,3,4,5,6,7])
    data['loan_purpose'] = data['loan_purpose'].replace(['P','C','N','NA'],[1,2,3,4])
    data['super_conforming_flag'] = data['super_conforming_flag'].replace(['Y','N'],[0,1])
    
    return data

In [4]:
def changedatatype(data):
    data[['credit_score','msa','no_of_units','mortage_insurance_pct','cltv','dti_ratio','original_ltv','zipcode','number_of_borrowers']]=data[['credit_score','msa','no_of_units','cltv','mortage_insurance_pct','dti_ratio','original_ltv','zipcode','number_of_borrowers']].astype('int64')
    data[['fthb_flag','occupancy_status','channel']] = data[['fthb_flag','occupancy_status','channel']].astype('int64')
    data[['ppm_flag','prop_type','loan_purpose','super_conforming_flag']]= data[['ppm_flag','prop_type','loan_purpose','super_conforming_flag']].astype('int64')
    data[['product_type','property_state']] = data[['product_type','property_state']].astype('str')
    data[['loan_seq_number','sellers_name','servicer_name']] = data[['loan_seq_number','sellers_name','servicer_name']].astype('str')
    return data

In [5]:
train_data = pd.read_csv('./train_Q12005.csv')
test_data = pd.read_csv('./test_Q22005.csv')

data_train = cleaningdata(train_data)
data_test = cleaningdata(test_data)
data_train = convertnumbersdata(data_train)
data_test = convertnumbersdata(data_test)
data_train = changedatatype(data_train)
data_test = changedatatype(data_test)

In [6]:
from tpot import TPOTRegressor

C:\Users\Komal\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\Komal\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Komal\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [7]:
y1 = data_train.original_int_rt

In [8]:
x1 = data_train

In [9]:
x1.drop(['loan_seq_number','property_state','sellers_name','servicer_name','super_conforming_flag' ,'product_type', 'original_int_rt','zipcode','msa'], axis =1, inplace = True)

In [10]:
y2 = data_test.original_int_rt

In [11]:
x2 = data_test

In [12]:
x2.drop(['loan_seq_number','property_state','sellers_name','servicer_name','super_conforming_flag' ,'product_type', 'original_int_rt','zipcode','msa'], axis =1, inplace = True)

In [ ]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(x1, y1)  
print(tpot.score(x2, y2))


C:\Users\Komal\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Komal\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Generation 1 - Current best internal CV score: -0.08029201091216487
Generation 2 - Current best internal CV score: -0.07384083598147335
